In [18]:
# Data Preprocessing

import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder 

print('pandas ', pd.__version__)
print('numpy ', np.__version__)
print('sklearn', sk.__version__)

pandas  1.0.5
numpy  1.18.5
sklearn 0.23.1


In [11]:
data = pd.read_csv('data/WA_Fn-UseC_-HR-Employee-Attrition.csv')
data.shape

(1470, 35)

In [12]:
y = data['Attrition']
print(y)

0       Yes
1        No
2       Yes
3        No
4        No
       ... 
1465     No
1466     No
1467     No
1468     No
1469     No
Name: Attrition, Length: 1470, dtype: object


In [13]:
X = data.drop(columns =['Attrition'])
print(X)

      Age     BusinessTravel  DailyRate              Department  \
0      41      Travel_Rarely       1102                   Sales   
1      49  Travel_Frequently        279  Research & Development   
2      37      Travel_Rarely       1373  Research & Development   
3      33  Travel_Frequently       1392  Research & Development   
4      27      Travel_Rarely        591  Research & Development   
...   ...                ...        ...                     ...   
1465   36  Travel_Frequently        884  Research & Development   
1466   39      Travel_Rarely        613  Research & Development   
1467   27      Travel_Rarely        155  Research & Development   
1468   49  Travel_Frequently       1023                   Sales   
1469   34      Travel_Rarely        628  Research & Development   

      DistanceFromHome  Education EducationField  EmployeeCount  \
0                    1          2  Life Sciences              1   
1                    8          1  Life Sciences             

In [28]:
def transform(feature):
    le=LabelEncoder()
    X[feature]=le.fit_transform(X[feature])
    print(le.classes_)

In [29]:
cat_df=X.select_dtypes(include='object')
cat_df.columns

Index(['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole',
       'MaritalStatus', 'Over18', 'OverTime'],
      dtype='object')

In [30]:
for col in cat_df.columns:
    transform(col)

['Non-Travel' 'Travel_Frequently' 'Travel_Rarely']
['Human Resources' 'Research & Development' 'Sales']
['Human Resources' 'Life Sciences' 'Marketing' 'Medical' 'Other'
 'Technical Degree']
['Female' 'Male']
['Healthcare Representative' 'Human Resources' 'Laboratory Technician'
 'Manager' 'Manufacturing Director' 'Research Director'
 'Research Scientist' 'Sales Executive' 'Sales Representative']
['Divorced' 'Married' 'Single']
['Y']
['No' 'Yes']


In [31]:
X.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,2,1102,2,1,2,1,1,1,2,...,1,80,0,8,0,1,6,4,0,5
1,49,1,279,1,8,1,1,1,2,3,...,4,80,1,10,3,3,10,7,1,7
2,37,2,1373,1,2,2,4,1,4,4,...,2,80,0,7,3,3,0,0,0,0
3,33,1,1392,1,3,4,1,1,5,4,...,3,80,0,8,3,3,8,7,3,0
4,27,2,591,1,2,1,3,1,7,1,...,4,80,1,6,3,3,2,2,2,2


In [32]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

X_auto = SelectKBest(chi2, k =22).fit_transform(X,y)

In [33]:
X_auto.shape

(1470, 22)

In [34]:
scaler=StandardScaler()
scaled_df=scaler.fit_transform(X_auto)
X_auto=scaled_df


In [35]:
print(X_auto[0])

[ 0.4463504   0.74252653 -1.01090934 -1.70128295 -0.66053067  0.37967213
 -0.05778755  1.03271569  1.15325359  1.23682046 -0.10834951  0.72601994
  2.12513592  1.59174553 -1.58417824 -0.93201439 -0.42164246 -2.17198183
 -0.16461311 -0.0632959  -0.67914568  0.24583399]
